In [4]:
import transformers
import torch
import gc

In [5]:
model_name = "teknium/OpenHermes-2.5-Mistral-7B"

def delete_model():
    global model
    global tokenizer
    model = None
    tokenizer = None
    gc.collect()
    torch.cuda.empty_cache()

def load_model(model_name: str):
    tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)

    with torch.device("cuda:0"):
        model = transformers.AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16).eval()
    
    return tokenizer, model

tokenizer, model = load_model(model_name)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [15]:
import inspect
import json
from typing import get_type_hints

class Article:
    pass

class Weather:
    pass

class Directions:
    pass

def calculate_mortgage_payment(loan_amount: int, interest_rate: float, loan_term: int) -> float:
    """Get the monthly mortgage payment given an interest rate percentage."""
    
    # TODO: you must implement this to actually call it later
    pass

def get_article_details(title: str, authors: list[str], short_summary: str, date_published: str, tags: list[str]) -> Article:
    '''Get article details from unstructured article text.
date_published: formatted as "MM/DD/YYYY"'''
    
    # TODO: you must implement this to actually call it later
    pass

def get_weather(zip_code: str) -> Weather:
    """Get the current weather given a zip code."""
    
    # TODO: you must implement this to actually call it later
    pass

def get_directions(start: str, destination: str) -> Directions:
    """Get directions from Google Directions API.
start: start address as a string including zipcode (if any)
destination: end address as a string including zipcode (if any)"""
    
    # TODO: you must implement this to actually call it later
    pass

def get_type_name(t):
    name = str(t)
    if "list" in name or "dict" in name:
        return name
    else:
        return t.__name__

def serialize_function_to_json(func):
    signature = inspect.signature(func)
    type_hints = get_type_hints(func)

    function_info = {
        "name": func.__name__,
        "description": func.__doc__,
        "parameters": {
            "type": "object",
            "properties": {}
        },
        "returns": type_hints.get('return', 'void').__name__
    }

    for name, _ in signature.parameters.items():
        param_type = get_type_name(type_hints.get(name, type(None)))
        function_info["parameters"]["properties"][name] = {"type": param_type}

    return json.dumps(function_info, indent=2)

print(serialize_function_to_json(get_article_details))

{
  "name": "get_article_details",
  "description": "Get article details from unstructured article text.\ndate_published: formatted as \"MM/DD/YYYY\"",
  "parameters": {
    "type": "object",
    "properties": {
      "title": {
        "type": "str"
      },
      "authors": {
        "type": "list[str]"
      },
      "short_summary": {
        "type": "str"
      },
      "date_published": {
        "type": "str"
      },
      "tags": {
        "type": "list[str]"
      }
    }
  },
  "returns": "Article"
}


In [16]:
import xml.etree.ElementTree as ET
import re

def extract_function_calls(completion):
    completion = completion.strip()
    pattern = r"(<multiplefunctions>(.*?)</multiplefunctions>)"
    match = re.search(pattern, completion, re.DOTALL)
    if not match:
        return None
    
    multiplefn = match.group(1)
    root = ET.fromstring(multiplefn)
    functions = root.findall("functioncall")
    return [json.loads(fn.text) for fn in functions]

In [17]:
def generate_hermes(prompt):
    fn = """{"name": "function_name", "arguments": {"arg_1": "value_1", "arg_2": value_2, ...}}"""
    prompt = f"""<|im_start|>system
You are a helpful assistant with access to the following functions:

{serialize_function_to_json(get_weather)}

{serialize_function_to_json(calculate_mortgage_payment)}

{serialize_function_to_json(get_directions)}

{serialize_function_to_json(get_article_details)}

To use these functions respond with:
<multiplefunctions>
    <functioncall>{fn}</functioncall>
    <functioncall>{fn}</functioncall>
    ...
</multiplefunctions>

Edge cases you must handle:
- If there are no functions that match the user request, you will respond politely that you cannot help.<|im_end|>
<|im_start|>user
{prompt}<|im_end|>
<|im_start|>assistant"""

    tokens = tokenizer(prompt, return_tensors="pt").to(model.device)
    input_size = tokens.input_ids.numel()
    with torch.inference_mode():
        generated_tokens = model.generate(**tokens, use_cache=True, do_sample=True, temperature=0.2, top_p=1.0, top_k=0, max_new_tokens=512, eos_token_id=tokenizer.eos_token_id, pad_token_id=tokenizer.eos_token_id)

    return tokenizer.decode(generated_tokens.squeeze()[input_size:], skip_special_tokens=True)

In [22]:
prompts = [
    "What's the weather in 10001?",
    "Determine the monthly mortgage payment for a loan amount of $200,000, an interest rate of 4%, and a loan term of 30 years.",
    "What's the current exchange rate for USD to EUR?"
]

for prompt in prompts:
    completion = generate_hermes(prompt)
    functions = extract_function_calls(completion)

    if functions:
        print(functions)
    else:
        print(completion.strip())
    print("="*100)

[{'name': 'get_weather', 'arguments': {'zip_code': '10001'}}]
[{'name': 'calculate_mortgage_payment', 'arguments': {'loan_amount': 200000, 'interest_rate': 0.04, 'loan_term': 30}}]
I'm sorry, but I don't have a function to get the current exchange rate. I can only help with the functions provided.


In [23]:
prompts = [
    "What's the weather in 05751?",
    "I'm planning a trip to Killington, Vermont (05751) from Hoboken, NJ (07030). Can you get me weather for both locations and directions?",
    "What's the current exchange rate for USD to EUR?"
]

for prompt in prompts:
    completion = generate_hermes(prompt)
    functions = extract_function_calls(completion)

    if functions:
        for function in functions:
            print(function["name"])
            print(function["arguments"])
    else:
        print(completion.strip())

    print("="*100)

get_weather
{'zip_code': '05751'}
get_weather
{'zip_code': '05751'}
get_weather
{'zip_code': '07030'}
get_directions
{'start': 'Hoboken, NJ 07030', 'destination': 'Killington, VT 05751'}
I'm sorry, but I don't have a function to get the current exchange rate. I can only help with the functions provided.
